# Progetto: Excel per HR

Lettura unput utente:
- Leggere la data come input()

Foglio "Candidati":
- Filtrare per "Data invio CV al cliente" == data da input utente.
- Leggere tutti i valori delle colonne per ciascuna (attenzione, i dati, non le formule con CERCA.VERT).

Cartella con i CV:
- Nella cartella indicata, cerca tutti i file di CV che corrispondono alle coppie "Nome" e "Cognome" dei candidati selezionati e estensione .PDF o .DOCX
- (Se trova dei file diversi da PDF o .DOCX ma che corrispondono al pattern di ricerca, avvisa l'utente che ci sono file non compatibili con l'invio e di provare a forzarne la conversione in PDF o DOCX.)

Report finale:
- Creazione di un nuovo file Excel che riporti solo le informazioni dei candidati selezionati in base alla "Data invio CV al cliente.

In [1]:
from pprint import pprint
import datetime
import openpyxl


book = openpyxl.load_workbook('./DB C-Lab (HRR) test.xlsx', data_only=True)

target_sheet = 'Candidati'
target_col_label = 'Data invio CV al cliente'
target_col_num = None
# data_invio = input('Inserire la "Data invio CV al cliente" in formato dd/mm/yyyy:')
# target_date = datetime.datetime.strptime(data_invio, '%d/%m/%Y')
target_date = datetime.date.fromisoformat('2023-05-15')

# controllo che il foglio "Candidati" esista
if target_sheet not in book.sheetnames:
    raise ValueError('Il foglio "{target_sheet}" non è stato trovato nella '
                     f'cartella di lavoro.')

foglio_candidati = book[target_sheet]

# individuo il numero di colonna della data oggetto del filtro
for col_num in range(1, foglio_candidati.max_column):
    column_name = foglio_candidati.cell(1, col_num).value
    if column_name == target_col_label:
        target_col = col_num

# controllo che la colonna con la data oggetto del filtro esista
if not target_col:
    raise ValueError(f'La colonna "{target_col_label}" non è stata trovata '
                     f'nel foglio "{target_sheet}".')

# creo un contenitore per i risultati del foglio "Candidati"
candidati_filtrati = []

# per ciascuna riga
for row_num in range(1, foglio_candidati.max_row):
    # ottiene il valore del campo con la data target
    date_cell = foglio_candidati.cell(row_num, target_col).value
    # se è la prima riga oppure se il valore ottenuto è di tipo data e se la data corrisponde
    if row_num == 1 or (isinstance(date_cell, datetime.datetime) and date_cell.date() == target_date):
        # copia l'intera riga
        record = []
        for col_num in range(1, foglio_candidati.max_column):
            record.append(foglio_candidati.cell(row_num, col_num).value)

        candidati_filtrati.append(record)

# pprint(candidati_filtrati)

# creo il file di output
book_transfer = openpyxl.Workbook()
foglio_transfer = book_transfer.active
foglio_transfer.title = 'Candidati'

# compilo il foglio di output
for row_num, row in enumerate(candidati_filtrati):
    for col_num, cell in enumerate(row):
        # se il valore che sta per scrivere è una data, la converte in stringa
        if isinstance(cell, datetime.datetime):
            cell = cell.strftime('%d/%m/%Y')
        foglio_transfer.cell(row_num+1, col_num+1).value = cell

# salvo il file di output
book_transfer.save('./DB C-Lab (Transfr) test.xlsx')

/Users/knox/Repositories/python_training/python_corso_base/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/knox/Repositories/python_training/python_corso_base/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
